# Hathi Parse MARC add ISBN and OCLC

This script expects to work on a TSV that has MARC XML in a `hathi_marc` column, likely added by the `download_hathi_marc` script previously. It will add oclc and isbn number from the MARC21 fields 001 and 020 adding `isbn` and `oclc` to the file.

In [29]:
import pandas as pd
import pymarc
import io


In [30]:
path_to_tsv = "/Users/m/Downloads/data-tmp/nyt_hardcover_fiction_bestsellers-hathitrust_metadata.tsv"


In [31]:
def add_auth_name(d):

    # do we have some marc data?
    if type(d['hathi_marc']) == str:   
        # the pymarc library expects a file to open, we don't have files we have strings, 
        # so make a file like object and put our string into it so we can parse it
        with io.StringIO() as f:
            f.write(d['hathi_marc'])
            f.seek(0)
            # parse it, its returns a list of records, but we only have one, so take the 0 index
            record = pymarc.marcxml.parse_xml_to_array(f)[0]

            if '001' in record:
                    
                if 'ocm' in record['001'].value():
                    d['oclc'] = record['001'].value().replace("ocm",'')
                if 'ocn' in record['001'].value():
                    d['oclc'] = record['001'].value().replace("ocn",'')
                if 'on' in record['001'].value():
                    d['oclc'] = record['001'].value().replace("on",'')                    

            if '035' in record:

                for f in record.get_fields('035'):
                    if 'a' in f:
                        if 'OCoLC' in f['a']:
                            d['oclc'] = f['a'].split(")")[1]
                            

            if '020' in record:
                if 'a' in record['020']:
                    i = record['020']['a']
                    i = i.split("(")[0].strip()
                    i = i.split(":")[0].strip()
                    d['isbn'] = i

    else:
        print("No MARC data to parse:",d)

    return d

In [32]:
# load the tsv
df = pd.read_csv(path_to_tsv, sep='\t', header=0, low_memory=False)
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)

# run our function over all of them
df = df.apply(lambda d: add_auth_name(d),axis=1 )  
# # overwrite back out
df.to_csv(path_to_tsv, sep='\t')



No MARC data to parse: author                               Sherrilyn Kenyon
author_authorized_heading                         NaN
author_lccn                                       NaN
author_viaf                                       NaN
best_rank                                           4
debut_rank                                         12
first_week                                 2011-08-21
hathi_marc                                        NaN
hathi_rights                                      NaN
htid                         uc2.ark+=13960=t4gm8336z
isbn                                              NaN
oclc                                              NaN
oclc_classify                                     NaN
oclc_eholdings                                    NaN
oclc_holdings                                     NaN
oclc_owi                                          NaN
title                                     RETRIBUTION
title_id                                         3777
year 